Here we seek to utilise the AAVSO api to draw back a list of targets for a given night

At https://targettool.aavso.org/TargetTool/api
we read that we can set the parameters of our site and then request the priority targets for tonight

In [18]:
import requests
import xmltodict
import datetime
from dateutil import tz
import pytz
import xmltodict
from astropy.time import Time
import ephem

entry = requests.get("https://targettool.aavso.org/TargetTool/api/v1/telescope",auth=(API_KEY,"api_token"))
print(entry.json())


{'latitude': -35.351835, 'longitude': 149.030122, 'targetaltitude': 20.0, 'sunaltitude': -5.0}


In [2]:
entry = requests.get("https://targettool.aavso.org/TargetTool/api/v1/nighttime",auth=(API_KEY,"api_token"))
print(entry.json())


{'status': 'SUN_IS_UP', 'sunset': 1719559606, 'sunrise': 1719607663}


In [3]:
obs_sections = ['eb'] #ac,ep,cv,eb,spp,lpv,yso,het,misc,all.

In [4]:
entry = requests.get("https://targettool.aavso.org/TargetTool/api/v1/targets",\
                     auth=(API_KEY,"api_token"),params={'observable':['True'],'obs_section':[obs_sections]})
targets = entry.json()

Here is what is available

Star Name: ASAS J081631-3314.7
{'VOTABLE': {'@version': '1.0', 'RESOURCE': {'DESCRIPTION': 'International Variable Star Index (VSX) Query Results', 'TABLE': {'FIELD': [{'@id': 'auid', '@name': 'AUID'}, {'@id': 'name', '@name': 'Name'}, {'@id': 'const', '@name': 'Const'}, {'@id': 'radec2000', '@name': 'Coords(J2000)'}, {'@id': 'varType', '@name': 'VarType'}, {'@id': 'maxMag', '@name': 'MaxMag'}, {'@id': 'maxPass', '@name': 'MaxMagPassband'}, {'@id': 'minMag', '@name': 'MinMag'}, {'@id': 'minPass', '@name': 'MinMagPassband'}, {'@id': 'epoch', '@name': 'Epoch'}, {'@id': 'novaYr', '@name': 'NovaYear'}, {'@id': 'period', '@name': 'Period'}, {'@id': 'riseDur', '@name': 'RiseDuration'}, {'@id': 'specType', '@name': 'SpecType'}, {'@id': 'disc', '@name': 'Discoverer'}], 'DATA': {'TABLEDATA': {'TR': {'TD': [None, 'ASAS J081631-3314.7', 'Pup', '124.12916667,-33.24397222', 'EA', '10.310', 'V', '10.690', 'V', '53543.30000', None, '189.28', '2', None, None]}}}}}}}

In [5]:
targets['targets']
for star_info in targets['targets']:
    star_name = star_info.get('star_name')
    if star_name:
        print(f"Star Name: {star_name}")
        # for each star name look up vsx to find the ephemeris
        fmt_star_name = star_name.replace(" ","+")
        vsx_req = requests.get(f"https://www.aavso.org/vsx/index.php?view=query.votable&ident={fmt_star_name}&format=xml")
        json_data = xmltodict.parse(vsx_req.content)
        const = json_data['VOTABLE']['RESOURCE']['TABLE']['DATA']['TABLEDATA']['TR']['TD'][2]
        radec = json_data['VOTABLE']['RESOURCE']['TABLE']['DATA']['TABLEDATA']['TR']['TD'][3]
        vartyp = json_data['VOTABLE']['RESOURCE']['TABLE']['DATA']['TABLEDATA']['TR']['TD'][4]
        vmin = json_data['VOTABLE']['RESOURCE']['TABLE']['DATA']['TABLEDATA']['TR']['TD'][5]
        vmax = json_data['VOTABLE']['RESOURCE']['TABLE']['DATA']['TABLEDATA']['TR']['TD'][7]
        photsys = json_data['VOTABLE']['RESOURCE']['TABLE']['DATA']['TABLEDATA']['TR']['TD'][6]
        epoch = json_data['VOTABLE']['RESOURCE']['TABLE']['DATA']['TABLEDATA']['TR']['TD'][9]
        period = json_data['VOTABLE']['RESOURCE']['TABLE']['DATA']['TABLEDATA']['TR']['TD'][11]
        print(f"{const}, {radec}, {vartyp}, {vmin}, {vmax}, {epoch}, {period}")

Star Name: ASAS J081631-3314.7
Pup, 124.12916667,-33.24397222, EA, 10.310, 10.690, 53543.30000, 189.28
Star Name: AW Col
Col, 91.29700000,-32.73091667, EA, 8.000, 8.630, 52939.82500, 10.3175
Star Name: AZ Cir
Cir, 219.59333000,-66.24567000, EA, 14.500, 15.500, 38547.29000, 0
Star Name: BB Cha
Cha, 195.43187500,-80.68475000, EA, 12.750, 13.670, 58172.67700, 13.59454
Star Name: BP Hyi
Hyi, 31.94021000,-73.49608000, EA, 15.300, 16.200, 0.00000, 0
Star Name: BV Ant
Ant, 165.35617000,-37.17169000, EA, 11.520, 12.240, 53447.70500, 3.59426
Star Name: CU Mus
Mus, 171.77292000,-74.47433000, EA, 14.500, 15.500, 38546.24000, 0
Star Name: Dauban V239
Vul, 294.03497000,25.75285000, EA, 13.300, 14.600, 58357.76820, 2.911974
Star Name: Dauban V241
Vul, 295.99217000,26.34973000, EA, 13.700, 15.300, 58658.90090, 3.48806
Star Name: Dauban V242
Lyr, 286.38033000,28.92342000, EA, 14.000, 15.700, 57438.42500, 3.266221
Star Name: Dauban V243
Vul, 305.82983000,21.84789000, EA, 12.700, 13.800, 0.00000, 0.0000

KeyboardInterrupt: 

In [6]:

# Make an observer
wombat = ephem.Observer()

# Set the date and time 
wombat.date = "2024-06-28"

# Location 
wombat.lon = str(148.12899)
wombat.lat = str(-35.282)

# Elevation 
wombat.elev = 500

# To get U.S. Naval Astronomical Almanac values, use these settings
wombat.pressure = 0
wombat.horizon = '-0:34'

# Calculate sunrise, solar noon, and sunset
sunrise = wombat.next_rising(ephem.Sun())
#noon = wombat.next_transit(ephem.Sun(), start=sunrise)
sunset = wombat.next_setting(ephem.Sun())

# Relocate the horizon to get twilight times
wombat.horizon = '-12'  # -6=civil twilight, -12=nautical, -18=astronomical
beg_twilight = wombat.next_rising(ephem.Sun(), use_center=True)
end_twilight = wombat.next_setting(ephem.Sun(), use_center=True)

# Print the results
#print(f"Noon: {noon}")
print(f"Sunset: {sunset}")
print(f"End nautical twilight: {end_twilight}")
print(f"Begin nautical twilight: {beg_twilight}")
print(f"Sunrise: {sunrise}")
print(f"MJD start: {(end_twilight+2415020-2400000.5 ):.6f}")
print(f"MJD end: {(beg_twilight+2415020-2400000.5 ):.6f}")

Sunset: 2024/6/28 07:04:40
End nautical twilight: 2024/6/28 08:04:42
Begin nautical twilight: 2024/6/28 20:17:01
Sunrise: 2024/6/28 21:17:01
MJD start: 60489.336598
MJD end: 60489.845148


so we want to look for variables that have minima in the above range

In [22]:
mjd_start = (end_twilight+2415020-2400000.5 )
mjd_end = (beg_twilight+2415020-2400000.5 )
(ep0, p) = (53389.80500, 0.90819)
ep = ep0
while (ep<mjd_start):
    ep +=p
ep -= 0.5 # somewhere there is a jd and not a mjd
print(f"Minimum at: {ep}")
if (ep > mjd_start and ep < mjd_end):
    print('well placed for observing tonight')
    jd_value = ep + 2400000.5 ### this is correct

    # Create an astropy Time object from JD
    t = Time(jd_value, format='jd')
    # Convert to datetime (UTC)
    utc_datetime = t.to_datetime(tz.UTC)
    print(f"UTC Date and Time: {utc_datetime}")
    target_timezone = pytz.timezone('Australia/Canberra')
    local_datetime = utc_datetime.astimezone(target_timezone)
    print(f"Local datetime (Canberra): {local_datetime.isoformat()}")

Minimum at: 60489.534420015036
well placed for observing tonight
UTC Date and Time: 2024-06-28 12:49:33.889281+00:00
Local datetime (Canberra): 2024-06-28T22:49:33.889281+10:00
